<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Burp_Fuzzing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Extending Burp Proxy__*

If you've ever tried hacking a web application, you've likely used __Burp Suite__ to perform spidering, proxy browser traffic, and carry out other attacks. Burp Suite also allows you to create your own tooling, called __extensions__. Using Python, Ruby, or pure Java, you can add panels in the Burp GUI and build automation techniques into Burp Suite. We'll take advantage of this feature to write some handy tooling for performing attacks and extended reconnaissance. The first extension will use an intercepted and extended reconnaissance. The first extension will use an intercepted HTTP request from Burp Proxy as a seed for a mutation fuzzer that runs in Burp Intruder. The second extension will communicate with the Microsoft Bing API to show us all virtual hosts located on the same IP address as a target site, as well as any subdomains detected for the target domain. Finally, we'll build an extension to create a word list from a target website that you can use in a brute-force password attack.

This chapter assumes that you've played with Brup before and know how to trap requests with the Proxy tool, as well as how to send a trapped request to Burp Intruder. If you need a tutorial on how to do these tasks, visit PortSwigger Web Security (_http://www.portswigger.net/_) to get started.

We have to admit that when we first started exploring the Burp Extender API, it took us some time to understand how it worked. We found it a bit confusing, as we're pure Python guys and have limited Java development experience. But we found a number of extensions on the Burp website that taught us how other folks had developed extensions. We used that prior art to help us understand how to begin implementing our own code. This chapter will cover some basics on extending functionality, but we'll also show you how to use the API documentation as a guide.

### *__Setting Up__*

Burp Suite comes installed by default on Kali Linux. If you're using a different machine, download Burp from _http://www.portswigger.net/_ and set ip up.

As sad as it makes us to admit this, you'll require a modern Java installation. Kali Linux has one installed. If you're on a different platform, use your system's installation method (such as apt, yum, or rpm) to get one. Next, install __Jython__, a Python 2 implementation written in Java. Up until now, all of our code has used Python 3 syntax, but in this chapter we'll revert to Python 2, since that's what Jython expects. You can find this JAR file on the official site, _https://www.jython.org/download.html__. Select the Jython 2.7 Standalone Installer. Save the JAR file to an easy to remember location, such as your Desktop.

Next, either double-click the Burp icon on your Kali machine or run Burp from the command line:
```
#> java -XX:MaxPermSize=1G -jar burpsuite_pro_v1.6.jar
```
This will fire up Burp, and you should see its graphical user interface (GUI) full of wonderful tabs, as shown in _Figure 6-1_.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Burp_Suite_GUI_Loaded_Properly.png?raw=true" alt="Burp Suite GUI Loaded Properly" width="70%">
<p style="text-align:center"><strong><em>Figure 6-1</strong></em></p>
</div>

Now let's point Burp at our Jython interpreter. Click the __Extender__ tab and then click the __Options__ tab. In the Python Environment section, select the location of your Jython JAR file, as shown in _Figure 6-2_. You can leave the rest of the options alone. We're ready to start coding our first extension. Let's get rocking!

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Configuring_The_Jython_Interpreter_Location.png?raw=true" alt="Configuring The Jython Interpreter Location" width="70%">
<p style="text-align:center"><strong><em>Figure 6-2</strong></em></p>
</div>

### *__Burp Fuzzing__*

At some point in your career, you may find yourself attacking a web application or service that doesn't allow you to use traditional web application assessment tools. For example, the application might use too many parameters, or it may be obfuscated in some way that makes performing a manual test far too time consuming. We've been guilty of running standard tools that can't deal with strange protocols, or even JSON in a lot of cases. This is where you'll find it useful to establish a solid baseline of HTTP traffic, including authentication cookies, while passing off the body of the request to a custom fuzzer. This fuzzer can then manipulate the payload in any way you choose. We'll work on our first Burp extension by creating the world's simplest web application fuzzer, which you can then expand into something more intelligent.

Burp has a number of tools you can use when you're performing web application tests. Typically, you'll trap all requests using the _Proxy_, and when you see an interesting one, you'll send it to another Burp tool. A common technique is to send them to the _Repeater_ tool, which lets you replay web traffic as well as manually modify any interesting spots. To perform more automated attacks in query parameters, you can send a request to the _Intruder_ tool, which attempts to automatically figure out which areas of the web traffic you should modify and then allows you to use a variety of attacks to try to elicit error messages or tease out vulnerabilities. A Burp extension can interact in numerous ways with the Burp suite of tools. In our case, we'll bolt additional functionality directly onto the Intruder tool.

Our first instinct is to take a look at the Burp API documentation to determine what Burp classes we need to extend in order to write our custom extension. You can access this documentation by clicking the __Extender__ tab and then clicking the __APIs__ tab. The API can look a little daunting because it's very Java-y. But notice that the Burp developers have aptly named each class, making it easy to figure out where we want to start. In particular, because we're trying to fuzz web requests during an Intruder attack, we might want to focus on the __IIntruderPayloadGeneratorFactory__ and __IIntruderPayloadGenerator__ classes. Let's take a look at what the documentation says for the __IIntruderPayloadGeneratorFactory__ class:
```
/**
 * Extensions can implement this interface and then call
 * IBurpExtenderCallbacks.registerIntruderPayloadGeneratorFactory() [1]
 * to register a factory for custom Intruder payloads.
 */

public interface IIntruderPayloadGeneratorFactory
{
    /**
     * This method is used by Burp to obtain the name of the payload
     * generator. This will be displayed as an option within the
     * Intruder UI when the user selects to use extension-generated
     * payloads.
     *
     * @return The name of the payload generator.
     */
    String getGeneratorName(); [2]

    /**
     * This method is used by Burp when the user starts an Intruder
     * attack that uses this payload generator.
     *
     * @param attack
     * An IIntruderAttack object that can be queried to obtain details
     * about the attack in which the payload generator will be used.
     * @return A new instance of
     * IIntruderPayloadGenerator that will be used to generate
     * payloads for the attack
     */
    IIntruderPayloadGenerator createNewInstance(IIntruderAttack attack); [3]
}
```

The first bit of documentation __[1]__ tells how to correctly register our extension with Burp. We'll extend the main Burp class as well as the __IIntruderPayloadGeneratorFactory__ class. Next, we see that Burp expects two methods in our main class. Burp will call the __getGeneratorName__ method __[2]__ to retrieve the name of our extension, and we're expected to return a string. The __createNewInstance__ method __[3]__ expects us to return an instance of the __IIntruderPayloadGenerator__, a second class we'll have to create.

Now let's implement the actual Python code to meet these requirements. Then we'll figure out how to add the __IIntruderPayloadGenerator__ class. Open a new Python file, name it __bhp_fuzzer.py__, and punch out the following code:

In [ ]:
from burp import IBurpExtender #[1]
from burp import IIntruderPayloadGeneratorFactory
from burp import IIntruderPayloadGenerator

from java.util import List, ArrayList

import random

class BurpExtender(IBurpExtender, IIntruderPayloadGeneratorFactory): #[2]
    def registerExtenderCallbacks(self, callbacks):
        self._callbacks = callbacks
        self._helpers = callbacks.getHelpers()

        callbacks.registerIntruderPayloadGeneratorFactory(self) #[3]

        return

    def getGeneratorName(self):
        return "BHP Payload Generator" #[4]

    def createNewInstance(self, attack): #[5]
        return BHPFuzzer(self, attack)

This simple skeleton outlines what we need in order to satisfy the first set of requirements. We have to first import the __IBurpExtender__ class __[1]__, a requirement for every extension we write. We follow this up by importing the classes necessary for creating an Intruder payload generator. Next, we define the __BurpExtender__ class __[2]__, which extends the __IBurpExtender__ and __IIntruderPayloadGeneratorFactory__ classes. We then use the __registerIntruderPayloadGeneratorFactory__ method __[3]__ to register our class so that the Intruder toll is aware that we can generate payloads. Next, we implement the __getGeneratorName__ method __[4]__ to simply return the name of our payload generator. Finally, we implement the __createNewInstance__ method __[5]__, which receives the attack parameter and returns an instance of the __IIntruderPayloadGenerator__ class, which we called __BHPFuzzer__.

Let's have a peek at the documentation for the __IIntruderPayloadGenerator__ class so we know what to implement:
```
/**
 * This interface is used for custom Intruder payload generators.
 * Extensions that have registered an IIntruderPayloadGeneratorFactory
 * must return a new instance of this interface when required as 
 * part of a new Intruder attack
 */

public interface IIntruderPayloadGenerator
{
    /**
     * This method is used by Burp to determine whether the payload
     * generator is able to provide any further payloads.
     *
     * @return Extensions should return false when all the available
     * payloads have been used up, otherwise true
     */
    boolean hasMorePayloads(); [1]

    /**
     * This method is used by Burp to obtain the value of the next payload.
     *
     * @param baseValue The base value of the current payload position.
     * This value may be null if the concept of a base value is not
     * applicable (e.g. in a battering ram attack).
     * @return The next payload to use in the attack.
    byte[] getNextPayload(byte[] baseValue); [2]

    /**
     * This method is used by Burp to reset the state of the payload
     * generator so that the next call to getNextPayload() returns
     * the first payload again. This method will be invoked when an 
     * attack uses the same payload generator for more than one payload 
     * position, for example in a sniper attack.
    void reset(); [3]
}
```
Okay! Now we know we need to implement the base class, which needs to expose three methods. The first method, __hasMorePayloads__ __[1]__, is there to decide whether to continue sending mutated requests back to Burp Intruder. We'll use a counter to deal with this. Once the counter reaches the maximum level, we'll return _False_ to stop generating fuzzing cases. The __getNextPayload__ method __[2]__ will receive the original payload from the HTTP request that you trapped. Alternatively, if you selected multiple payload areas in the HTTP request, you'll receive only the bytes you plan to fuzz (more on this later). This method allows us to fuzz the original test case and then return it for Burp to send. The last method, __reset__ __[3]__, is there so that if we generate a known set of fuzzed requests, the fuzzer can iterate through those values for each payload position designated in the Intruder tab. Our fuzzer isn't so fussy, it will always just keep randomly fuzzing each HTTP request.

Now let's see how this looks when we implement it in Python. Add the following code to the bottom of __bhp_fuzzer.py__:

In [ ]:
class BHPFuzzer(IIntruderPayloadGenerator): #[1]
    def __init__(self, extender, attack):
        self._extender = extender
        self._helpers = extender._helpers
        self._attack = attack
        self.max_payloads = 10 #[2]
        self.num_iterations = 0

        return
    
    def hasMorePayloads(self): #[3]
        if self.num_iterations == self.max_payloads:
            return False
        else:
            return True

    def getNextPayload(self, current_payload): #[4]
        # Convert into a string
        payload = "".join(chr(x) for x in current_payload) #[5]

        # Call our simple mutator to fuzz the POST
        payload = self.mutate_payload(payload) #[6]

        # Increase the number of fuzzing attempts
        self.num_iterations += 1 #[7]

        return payload

    def reset(self):
        self.num_payloads = 0
        return

We start by defining a __BHPFuzzer__ class __[1]__ that extends the class __IIntruderPayloadGenerator__ class. We define the required class variables and then add the __max_payloads__ __[2]__ and __num_iterations__ variables used to let Burp know when we've finished fuzzing. You could, of course, let the extension run forever if you'd like, but for testing purposes, we'll set time limits. Next, we implement the __hasMorePayloads__ method __[3]__, which simply checks whether we've reached the maximum number of fuzzing iterations. You could modify this to continually run the extension by always returning _True_. The __getNextPayload__ method __[4]__ receives the orginal HTTP payload, and it's here that we'll be fuzzing. The __current_payload__ variable arrives as a byte array, so we convert this to a string __[5]__ and the pass it to the __mutate_payload__ fuzzing method __[6]__. We then increment the __num_iterations__ variable __[7]__ and return the mutated payload. Our last method is the __reset__ method, which returns without doing anything.

Now let's write the world's simplest fuzzing method, which you can modify to your heart's content. For instance, this method knows the value of the current payload, so if you have a tricky protocol that needs something special, like CRC checksum or a length field, you could perform those calculations inside this method before returning. Add the following code to __bhp_fuzzer.py__, inside the __BHPFuzzer__ class:

In [ ]:
def mutate_payload(self, original_payload):
    # Pick a simple mutator or even call an external script
    picker = random.randint(1, 3)

    # Select a random offset in the payload to mutate
    offset = random.randint(0, len(original_payload) - 1)

    front, back = original_payload[:offset], original_payload[offset:] #[1]

    # Random offset insert a SQL injection attempt
    if picker == 1:
        front += "'" #[2]
    # Jam an XSS attempt in
    elif picker == 2:
        front += "<script>alert('BHP!'):</script>" #[3]
    # Repeat a random chunk of the original payload
    elif picker == 3:
        chunk_length = random.randint(0, len(back) - 1) #[4]
        repeater = random.randint(1, 10)
        for _ in range(repeater):
            front += original_payload[:offset + chunk_length]
    
    return front + back #[5]

First, we take the payload and split it into two random-length chunks, __front__ and __back__ __[1]__. Then, we randomly pick from three mutators: A simple SQL injection test that adds a single-quote to the end of the __front__ chunk __[2]__, a cross-site scripting (_XSS_) test that adds a script tag to the end of the __front__ chunk __[3]__. and a mutator that selects a random chunk from the original payload, repeats it a random number of times, and adds the result to the end of the __front__ chunk __[4]__. Then, we add the __back__ chunk to the altered __front__ chunk to complete the mutated payload __[5]__. We now have a Burp Intruder extrension we can use. Let's take a look at how to load it.

### *__Kicking the Tires__*

First, we have to load the extension and make sure it contains no errors. Click the __Extender__ tab in Burp and then click the __Add__ button. A screen should appear, allowing you to point Burp at the fuzzer. Ensure that you set the same options as the ones shown in _Figure 6-3_.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Setting_Burp_To_Load_Our_Extension.png?raw=true" alt="Setting Burp To Load Our Extension" width="70%">
<p style="text-align:center"><strong><em>Figure 6-3</strong></em></p>
</div>

Click __Next__, and Burp should begin loading the extension. If there are errors, clock the __Errors__ tab, debug any typos, and then clock __Close__. Your Extender screen should now look like _Figure 6-4_.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Burp_Extender_Showing_That_Our_Extension_Is_Loaded.png?raw=true" alt="Burp Extender Showing That Our Extension Is Loaded" width="70%">
<p style="text-align:center"><strong><em>Figure 6-4</strong></em></p>
</div>

As you can see, our extension has loaded and Burp has identified the registered Intruder payload generator. We're now ready to leverage the extension in a real attack. Make sure your web browser is set to use Burp Proxy as a localhost proxy on port 8080. Now let's attack the same Acunetix web application from Chapter 5. Simply browse to _http://testphp.vulnweb.com/_.

As an example, the authors used the little search bar on their site to submit a search for the string _"test"_. _Figure 6-5_ shows how you can see this request in the HTTP history tab of the Proxy menu. Right-click the request to send it to Intruder.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Selecting_An_HTTP_Request_To_Send_To_Intruder.png?raw=true" alt="Selecting An HTTP Request To Send To Intruder" width="70%">
<p style="text-align:center"><strong><em>Figure 6-5</strong></em></p>
</div>

Now switch to the __Intruder__ tab and click the __Positions__ tab. A screen should appear, showing each query parameter highlighted. This is Burp's way of identifying the spots we should be fuzzing. You can try moving the payload delimiters around or selecting the entire payload to fuzz if you choose, but for now, let's let Burp decide what to fuzz. For clarity, see _Figure 6-6_, which shows how payload highlighting works.

Now click the __Payloads__ tab. In this screen, click the __Payload type__ drop-down and select __Extension-generated__. In the Payload Options section, click the __Select generator__ button and choose __BHP Payload Generator__ from the drop-down. Your Payload screen should now look like _Figure 6-7_.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Burp_Intruder_Highlighting_Payload_Parameters.png?raw=true" alt="Burp Intruder Highlighting Payload Parameters" width="70%">
<p style="text-align:center"><strong><em>Figure 6-6</strong></em></p>
</div>

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Using_Our_Fuzzing_Extension_As_A_Payload_Generator.png?raw=true" alt="Using Our Fuzzing Extension As A Payload Generator" width="70%">
<p style="text-align:center"><strong><em>Figure 6-7</strong></em></p>
</div>

Now we're ready to send requests. At the top of the Burp menu bar, click __Intruder__ and then select __Start Attack__. Burp should begin sending fuzzed requests, and soon you'll be able to quickly go through the results. When the authors ran the fuzzer, we received the output shown in _Figure 6-8_.

<div align="center" width="100%">
<img src="https://github.com/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Our_Fuzzer_Running_In_An_Intruder_Attack.png?raw=true" alt="Our Fuzzer Running In An Intruder Attack" width="70%">
<p style="text-align:center"><strong><em>Figure 6-8</strong></em></p>
</div>

As you can see from the bold warning in the response to request 7, we've discovered what appears to be a SQL injection vulnerability.

Even though we built this fuzzer for demonstration purposes only, you'll be suprised how effective it can be for getting a web application to ouput errors, disclose application paths, or generate behavior that lots of other scanners might miss. Most importantly, we managed to get our custom extension to work with Burp's Intruder attacks. Now let's create an extension that will help us perform extended reconnaissance against a web server.